#### Load all dependencies

In [108]:
import numpy as np
import pandas as pd 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchsummary import summary
from PIL import Image
from tqdm import tqdm    # for progress bar
import os

#### Set device

In [109]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
device

device(type='cuda')

#### Define Transform

In [110]:
train_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.Resize((224,254)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

#### Load data

In [111]:
train_data = ImageFolder('/kaggle/input/cat-dog-pandas/Cat-Dog_Pandas/Train',
                        transform = train_transform)
val_data = ImageFolder('/kaggle/input/cat-dog-pandas/Cat-Dog_Pandas/Valid',
                      transform = test_transform)

In [112]:
class_names = train_data.classes   # different classes in our datasets
class_names

['cat', 'dog', 'panda']

#### Data loader

In [113]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [114]:
dataiter = iter(train_loader)
features,label = next(dataiter)
print('number of data per batch: ',len(features))
print('number of label per batch: ',len(label))
print('labels : ',label.unique())

number of data per batch:  32
number of label per batch:  32
labels :  tensor([0, 1, 2])


#### ResNet50 Tensfor Learning technique

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    
# Unfreeze only deeper layers for fine-tuning
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

num_features = model.fc.in_features  
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.6),
    nn.Linear(512,128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.3),
    nn.Linear(128, 3)) # 3 classes dog, cat, pandas
model = model.to(device)

In [116]:
summary(model,input_size=(3,224,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 122]           9,408
       BatchNorm2d-2         [-1, 64, 112, 122]             128
              ReLU-3         [-1, 64, 112, 122]               0
         MaxPool2d-4           [-1, 64, 56, 61]               0
            Conv2d-5           [-1, 64, 56, 61]           4,096
       BatchNorm2d-6           [-1, 64, 56, 61]             128
              ReLU-7           [-1, 64, 56, 61]               0
            Conv2d-8           [-1, 64, 56, 61]          36,864
       BatchNorm2d-9           [-1, 64, 56, 61]             128
             ReLU-10           [-1, 64, 56, 61]               0
           Conv2d-11          [-1, 256, 56, 61]          16,384
      BatchNorm2d-12          [-1, 256, 56, 61]             512
           Conv2d-13          [-1, 256, 56, 61]          16,384
      BatchNorm2d-14          [-1, 256,

#### Loss and Optimizer

In [117]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=5e-4)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(train_loader),
    epochs=num_epochs
)

In [118]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        """
        Args:
            patience (int): How many epochs to wait after last improvement.
            min_delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


#### Training Loop

In [119]:
num_epochs = 30
early_stopping = EarlyStopping(patience=5, min_delta=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Wrap train_loader with tqdm
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Training")
    
    for images, labels in train_loader_tqdm:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Update tqdm bar postfix with running metrics
        train_loader_tqdm.set_postfix({
            'loss': f"{loss.item():.4f}"
        })

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] Validation")
    
    with torch.no_grad():
        for images, labels in val_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            val_loader_tqdm.set_postfix({
                'loss': f"{loss.item():.4f}"
            })

    val_loss /= total_val
    val_acc = correct_val / total_val

    scheduler.step()
    
    # Final log for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"⏹ Early stopping triggered at epoch {epoch+1}")
        break


Epoch [1/30] Validation: 100%|██████████| 10/10 [00:02<00:00,  4.66it/s, loss=0.6913]


Epoch [1/30] Train Loss: 0.9966, Train Acc: 0.6943 Val Loss: 0.8067, Val Acc: 0.9733


Epoch [2/30] Validation: 100%|██████████| 10/10 [00:02<00:00,  4.72it/s, loss=0.3666]


Epoch [2/30] Train Loss: 0.6762, Train Acc: 0.9505 Val Loss: 0.4362, Val Acc: 0.9900


Epoch [3/30] Validation: 100%|██████████| 10/10 [00:02<00:00,  4.52it/s, loss=0.3008]


...

Epoch [14/30] Train Loss: 0.3209, Train Acc: 0.9962 Val Loss: 0.3182, Val Acc: 0.9867


Epoch [15/30] Validation: 100%|██████████| 10/10 [00:02<00:00,  4.62it/s, loss=0.2974]


Epoch [15/30] Train Loss: 0.3169, Train Acc: 0.9990 Val Loss: 0.3163, Val Acc: 0.9867


Epoch [16/30] Validation: 100%|██████████| 10/10 [00:02<00:00,  4.47it/s, loss=0.2969]

Epoch [16/30] Train Loss: 0.3179, Train Acc: 0.9971 Val Loss: 0.3156, Val Acc: 0.9933
⏹ Early stopping triggered at epoch 16


#### Model Evaluation on Test data

In [123]:
# Test transform

test_data = ImageFolder('/kaggle/input/cat-dog-pandas/Cat-Dog_Pandas/Test',
                       transform=test_transform)
test_loader =DataLoader(test_data, batch_size=32, shuffle=False)

#### Model testing with data

In [124]:
correct, total = 0, 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

accuracy = correct / total
print(f"\n✅ Overall Test Accuracy: {accuracy*100:.2f}%")


✅ Overall Test Accuracy: 99.50%


In [126]:
# Add this to the end of your training script
torch.save(model.state_dict(), 'model.pth')